# Fulham v. Southampton - 2024-12-22 match analysis
*This notebook intends to walk through the theory and calculations I am currently using in my model to calculate probability of 1 x 2 result and a table of over / under goal projections. I will break the calculations out into utility functions stored in `../src/utils` for ease of use in future explorations. This is to document the calculations.*

### Instructions:

1. Ensure all cells in all notebooks in `/notebooks` directory have been run.
2. Define `home_side` and `away_side` in first cell. If making a comparison with odds from `03_retrieve_odds.ipynb` also enter matchday.
3. Click 'run all'
4. Probability for 1 x 2 and over / under can be viewed in the last cell
5. *Suggestion:* copy and paste notebook to `../output/< matchday_directory >/` and rename file. IMPORTANT: Update file path in lines 14 and 15 if needed if you make a copy.



### Import League Data & Define Sides

Define 'home_side' and 'away_side'
Ensure correct output



In [11]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

"""
BUG: if copying from `notebooks/03_analysis_template_notebook.ipynb` to a directory in output
for storage, ensure the file path for reading the csv will need to be updated from '../data/processed/home_table.csv'
to '../../data/processed/home_table.csv' if frames are run again.

TLDR
if you are in ./notebooks/03_analysis_tempalte.csv you may run this cell as is. If you are not, check that following
read_csv commands read from the correct directory.
"""
home_dataframe = pd.read_csv('../data/processed/home_table.csv')
away_dataframe = pd.read_csv('../data/processed/away_table.csv')

"""
!!! INPUT TEAMS & MATCHDAY HERE !!!

wrapped in quotes, eg.

home_side = "Bournemouth"
away_side = "West Ham"
matchday = "2024-12-16" # (optional)

If there is a ValueError,
(1) check spelling against the error message. Must be exact.
(2) check capitalization
(3) check that it is wrapped in quotes

!!! INPUT TEAMS & MATCHDAY HERE !!!
"""
home_side = "Fulham"
away_side = "Southampton"
matchday = "2024-12-22" # (optional)

# validate input
clubs = set(home_dataframe['Team'].values)
if home_side not in clubs:
    raise ValueError(f"Invalid entry for 'home_side'. Expected one of {clubs}, but received '{home_side}'")
elif away_side not in clubs:
    raise ValueError(f"Invalid entry for 'away_side'. Expected one of {clubs}, but received '{away_side}'")
else:
    print(f"Ready to calculate probability for {home_side} v. {away_side}")
    print("\nNOTEBOOK TITLE (copy and paste): \n")
    print(f"{home_side} v. {away_side} - {matchday} match analysis")

Ready to calculate probability for Fulham v. Southampton

NOTEBOOK TITLE (copy and paste): 

Fulham v. Southampton - 2024-12-22 match analysis


### Calculate Projected Goals

In [12]:
# 2. grab their gpm_scored and gpm_conceded
home_side_gpm_scored = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_scored"
].values[0]

home_side_gpm_conceded = home_dataframe.loc[
    home_dataframe["Team"] == home_side,
    "gpm_conceded"
].values[0]

print(home_side_gpm_scored, home_side_gpm_conceded)

away_side_gpm_scored = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_scored"
].values[0]

away_side_gpm_conceded = away_dataframe.loc[
    away_dataframe["Team"] == away_side,
    "gpm_conceded"
].values[0]

print(away_side_gpm_scored, away_side_gpm_conceded)

# 3.
epl_home_avg_gpm_scored = round(sum(home_dataframe["gpm_scored"].values) / 20, 2)
epl_home_avg_gpm_conceded = round(sum(home_dataframe["gpm_conceded"].values) / 20, 2)
print(epl_home_avg_gpm_scored, epl_home_avg_gpm_conceded)

# 4. create a ratio (attack and defense rating) of their gpm to the league average gpm
home_attack_rating = home_side_gpm_scored / epl_home_avg_gpm_scored
away_defense_rating = away_side_gpm_conceded / epl_home_avg_gpm_scored
away_attack_rating = away_side_gpm_scored / epl_home_avg_gpm_conceded
home_defense_rating = home_side_gpm_conceded / epl_home_avg_gpm_conceded

print(home_attack_rating, away_defense_rating, away_attack_rating, home_defense_rating)

# home projected goals
home_projected_goals = home_attack_rating * away_defense_rating * epl_home_avg_gpm_scored
# away projected goals
away_projected_goals = away_attack_rating * home_defense_rating * epl_home_avg_gpm_conceded
print("\n ================== projected_goals ==================== \n")
print(f"HOME {home_side} projected goals:", home_projected_goals)
print(f"AWAY {away_side} projected goals: ", away_projected_goals)

1.75 1.625
0.5 1.875
1.61 1.4
1.0869565217391304 1.1645962732919255 0.35714285714285715 1.1607142857142858

 ================== projected_goals ==================== 

HOME Fulham projected goals: 2.0380434782608696
AWAY Southampton projected goals:  0.5803571428571429


### Poisson Distribution of Goal Probabilities

In [13]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)
print(f"{home_side} v. {away_side}", exact_score_prob)

Fulham v. Southampton [[7.29193954e-02 1.48612898e-01 1.51439774e-01 1.02880281e-01
  5.24186215e-02 2.13662860e-02 7.25756996e-03 2.11303473e-03
  5.38307082e-04]
 [4.23192919e-02 8.62485570e-02 8.78891545e-02 5.97073061e-02
  3.04215214e-02 1.24000767e-02 4.21198256e-03 1.22631480e-03
  3.12410360e-04]
 [1.22801517e-02 2.50274830e-02 2.55035493e-02 1.73257808e-02
  8.82767363e-03 3.59823653e-03 1.22222708e-03 3.55850277e-04
  9.06547919e-05]
 [2.37562458e-03 4.84162618e-03 4.93372233e-03 3.35171354e-03
  1.70773448e-03 6.96087425e-04 2.36442739e-04 6.88400833e-05
  1.75373853e-05]
 [3.44677674e-04 7.02468085e-04 7.15830249e-04 4.86297724e-04
  2.47773976e-04 1.00994827e-04 3.43053082e-05 9.98795851e-06
  2.54448671e-06]
 [4.00072300e-05 8.15364741e-05 8.30874397e-05 5.64452715e-05
  2.87594794e-05 1.17226139e-05 3.98186613e-06 1.15931661e-06
  2.95342208e-07]
 [3.86974695e-06 7.88671253e-06 8.03673152e-06 5.45973608e-06
  2.78179488e-06 1.13388378e-06 3.85150741e-07 1.12136280e-07
  

### Under Market

In [14]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) <= t
    # print(mask)
    sums.append(exact_score_prob[mask].sum())

# print(sums)
under_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})
under_market_df["Implied Odds"] = 1 / under_market_df["Prob"]
print("~  ~ under market ~  ~\n", f"{home_side} v. {away_side}\n", under_market_df)

~  ~ under market ~  ~
 Fulham v. Southampton
    Goals      Prob  Implied Odds
0    0.5  0.072919     13.713773
1    1.5  0.263852      3.790009
2    2.5  0.513820      1.946207
3    3.5  0.731993      1.366134


In [15]:
thresholds = [0.5, 1.5, 2.5, 3.5]
sums = []

for t in thresholds:
    mask = np.add.outer(
        np.arange(
            exact_score_prob.shape[0]
        ),
        np.arange(
            exact_score_prob.shape[1]
        )
    ) >= t
    sums.append(exact_score_prob[mask].sum())

# print(sums)
over_market_df = pd.DataFrame({
    "Goals": thresholds,
    "Prob": sums
})

over_market_df["Implied Odds"] = 1 / over_market_df["Prob"]
print("~  ~ over market ~  ~\n", f"{home_side} v. {away_side}\n", over_market_df)

~  ~ over market ~  ~
 Fulham v. Southampton
    Goals      Prob  Implied Odds
0    0.5  0.926809      1.078972
1    1.5  0.735876      1.358924
2    2.5  0.485908      2.058003
3    3.5  0.267735      3.735032


### 1 x 2 Probability

In [16]:
# Poisson probability for home team goals 0 - 8
home_score_prob = []
away_score_prob = []
for i in range(9):
    prob = poisson.pmf(i, home_projected_goals)
    home_score_prob.append(prob)
for i in range(9):
    prob = poisson.pmf(i, away_projected_goals)
    away_score_prob.append(prob)

home_score_prob = np.array(home_score_prob)
away_score_prob = np.array(away_score_prob)
# print(home_score_prob)
# print(away_score_prob)

exact_score_prob = np.outer(away_score_prob, home_score_prob)

# print(exact_score_prob)
home_win_prob = np.sum(np.triu(exact_score_prob, k=1))
away_win_prob = np.sum(np.tril(exact_score_prob, k=-1))
draw_prob = np.trace(exact_score_prob)

# prepare data in readable dataframe
data = {
    "winner": [f"{home_side} (1)", "Draw (x)", f"{away_side} (2)"],
    "prob": [home_win_prob, draw_prob, away_win_prob],
}
result_df = pd.DataFrame(data)
result_df['implied_odds'] = 1 / result_df['prob']
print("      ~  ~ moneyline market ~  ~\n", result_df)

      ~  ~ moneyline market ~  ~
             winner      prob  implied_odds
0       Fulham (1)  0.717096      1.394513
1         Draw (x)  0.188283      5.311151
2  Southampton (2)  0.094349     10.598968


# Full Results Summary:

In [17]:
# helper function to translate discrepency between Understat and Odds-Api naming
def translate(string):

    translations = {
        "Brighton": "Brighton and Hove Albion",
        "Ipswich": "Ipswich Town",
        "Leicester": "Leicester City",
        "Tottenham": "Tottenham Hotspur",
        "West Ham": "West Ham United"
    }

    if string in translations:
        return translations[string]
    return string

home_side, away_side = translate(home_side), translate(away_side)

match_string = f"{home_side} v. {away_side}"

print(f"CALCULATED PROBABILITIES FOR {home_side} v. {away_side}")
print("\n\n        ~ ~ moneyline market ~ ~\n\n", result_df)
print("\n\n          ~ ~ over market ~ ~ \n\n", over_market_df)
print("\n\n          ~ ~ under market ~ ~ \n\n", under_market_df)

try:
    odds_df = pd.read_csv(f"../data/processed/odds/odds_matchday_{matchday}.csv")
    filtered_odds = odds_df.loc[odds_df["match"] == match_string]
    implied = {
        "1": 1 / filtered_odds["1"],
        "x": 1 / filtered_odds["x"],
        "2": 1 / filtered_odds["2"],
        "point": filtered_odds["ovr_und_point"],
        "over": 1 / filtered_odds["over"],
        "under": 1 / filtered_odds["under"]
    }
    implied_odds_df = pd.DataFrame(implied)

    print(f"\n ~~ MyBookie Odds for {match_string} ~~\n",filtered_odds.drop(columns=["match"]))
    print(f"\n ~~ MyBookie Odds IMPLIED for {match_string} ~~\n", implied_odds_df)
except:
    print(f"No data for sportsbook odds on matchday {matchday}. To view against real odds, run `notebooks/03_retrieve_odds.ipynb` ran correctly and ensure matchday is correct in the first cell of this notebook")


CALCULATED PROBABILITIES FOR Fulham v. Southampton


        ~ ~ moneyline market ~ ~

             winner      prob  implied_odds
0       Fulham (1)  0.717096      1.394513
1         Draw (x)  0.188283      5.311151
2  Southampton (2)  0.094349     10.598968


          ~ ~ over market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.926809      1.078972
1    1.5  0.735876      1.358924
2    2.5  0.485908      2.058003
3    3.5  0.267735      3.735032


          ~ ~ under market ~ ~ 

    Goals      Prob  Implied Odds
0    0.5  0.072919     13.713773
1    1.5  0.263852      3.790009
2    2.5  0.513820      1.946207
3    3.5  0.731993      1.366134

 ~~ MyBookie Odds for Fulham v. Southampton ~~
       1    x    2  ovr_und_point  over  under
2  1.38  4.9  6.9            3.5   2.3   1.57

 ~~ MyBookie Odds IMPLIED for Fulham v. Southampton ~~
           1         x         2  point      over     under
2  0.724638  0.204082  0.144928    3.5  0.434783  0.636943


# Conclusion and Pick
SAFE: Fulham win - Southampton in dreadful form, this bet is for that run of terrible form to continue.

### My Pick:

**Fullhgam @ 1.38** -- *1 unit*